In [2]:
import pandas as pd
import re

In [25]:
df = pd.read_csv('aln_1A_kmer.sam',skiprows = 2,delim_whitespace=True,header=None, index_col=None,names =  ['QNAME','FLAG','RNAME','POS','MAPQ','CIGAR','RNEXT','PNEXT','TLEN','SEQ','QUAL','M1','M2','M3','M4','M5','M6','M7','M8','M9'])
#df.to_csv('1A_sam_output.csv', index = False)
df = df.iloc[:, : 10]
df['protospacer'] = 'Null'
df

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer
0,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,125,37,25M,*,0,0,GCTTGAAGGAGATGGAGAGCTCGAT,Null
1,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,150,37,25M,*,0,0,CGGTCTCCTCTCTCTTGCTGAGTGG,Null
2,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,175,37,25M,*,0,0,CTAACTAGCTCGTAGTGTGGACTAT,Null
3,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,200,37,25M,*,0,0,GAGAGAGACGGAGCCATGGCATCCA,Null
4,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,Null
...,...,...,...,...,...,...,...,...,...,...,...
33579,A00351:76:HCLKMDSXX:3:2678:27950:32957_4:N:0:0...,16,Chr3_GTL1_protospacer_region,652,37,25M,*,0,0,TGAAGATGACGATGATGATGTGAGA,Null
33580,A00351:76:HCLKMDSXX:3:2678:27950:32957_4:N:0:0...,16,Chr3_GTL1_protospacer_region,627,37,25M,*,0,0,AAGAAATGCTTCTGCGGATCGAAGG,Null
33581,A00351:76:HCLKMDSXX:3:2678:27950:32957_4:N:0:0...,16,Chr3_GTL1_protospacer_region,602,37,25M,*,0,0,GTGAAGAAGAAGAAGAAGGCGAGGA,Null
33582,A00351:76:HCLKMDSXX:3:2678:27950:32957_4:N:0:0...,16,Chr3_GTL1_protospacer_region,577,37,25M,*,0,0,AAGAAAGAAAGAAAGGCACTGTGCT,Null


In [26]:
#check unique indel types
unique_labels = list(df.CIGAR.unique())
print(len(unique_labels))
def unwanted_labels(element):
    pattern = r"(\d+)([a-zA-Z]+)"
    match = re.search(pattern, element)
    if match:
        m = re.findall(pattern,element)
        if len(m) >1 :
            #print("Full match: % s" % (match.group(0)),m) 
            return element
        else:
            return 'wt'
    else:
        return False
unique_labels

36


['25M',
 '11M1I13M',
 '*',
 '18M1I6M',
 '13M1I11M',
 '5M1I19M',
 '19M1I5M',
 '17M1I7M',
 '8M1I16M',
 '3M1I21M',
 '6M1I18M',
 '12M1I12M',
 '10M1I14M',
 '9M1I15M',
 '12M1D13M',
 '15M1D10M',
 '10M2I13M',
 '13M2I10M',
 '2M1I22M',
 '4M1I20M',
 '15M1I9M',
 '12M2D13M',
 '10M2D15M',
 '16M1I8M',
 '3M1D22M',
 '7M1I17M',
 '4M1D21M',
 '5M1D20M',
 '21M1I3M',
 '14M1I10M',
 '20M1I4M',
 '7M1D18M',
 '9M2D16M',
 '17M1D8M',
 '6M1D19M',
 '13M1D12M']

In [27]:
wt_bank = {'p1':"CACAACGACG",'p2':"GCTACGTACG",'p3':"CGGGGAATCT",'p4':"GGTGGGCCCG",'p5':'CGCGAACGCG','p6':'CCCACGTTGC','p7':'CACATGTTTT'}


In [28]:
#protospacer PAM index (not python index, followed sam file index)
perfect_match_requirement = 25
p_bank = {'p1':238,'p2':744,'p3':941,'p4':1277,'p5':1366,'p6':1402,'p7':1497}
#+1 to make sure PAM is covered
acceptable_index_range = {key: {key+'start': val-perfect_match_requirement+1, key+'end': val-1}  for key, val in p_bank.items()}
acceptable_index_range

{'p1': {'p1start': 214, 'p1end': 237},
 'p2': {'p2start': 720, 'p2end': 743},
 'p3': {'p3start': 917, 'p3end': 940},
 'p4': {'p4start': 1253, 'p4end': 1276},
 'p5': {'p5start': 1342, 'p5end': 1365},
 'p6': {'p6start': 1378, 'p6end': 1401},
 'p7': {'p7start': 1473, 'p7end': 1496}}

In [29]:
#taken 10 bp
specific_index_range = {key: val-5 for key, val in p_bank.items()}
specific_index_range

{'p1': 233,
 'p2': 739,
 'p3': 936,
 'p4': 1272,
 'p5': 1361,
 'p6': 1397,
 'p7': 1492}

In [30]:
# from the start POS, find which protospacer the seq looked at

def protospacer_finder(x, range_dict):
    find = False
    for key,value_ in range_dict.items():
        #print(value_.values(), type(value_.values()))
        range_to_find = list(value_.values())
        #print(range_to_find)
        if x in range(range_to_find[0], range_to_find[1]):
            find = True
            return key
        else:
            pass
    if not find:
        #target seq not at protospacer region
        return find
#test
a = protospacer_finder(1368,acceptable_index_range) 
a

False

In [31]:
#select rows that may contain possible indels surrounding PAM
possible_indel_rows = df.loc[df['CIGAR'].apply(unwanted_labels) != False]
possible_indel_rows = possible_indel_rows.loc[possible_indel_rows['POS'].apply(protospacer_finder,args=(acceptable_index_range,)) != False]
possible_indel_rows.reset_index(drop=True, inplace=True)
possible_indel_rows['real_pam_seq'] = 'Null'
possible_indel_rows

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq
0,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,Null,Null
1,A00351:76:HCLKMDSXX:3:1104:19280:10520_1:N:0:0...,16,Chr3_GTL1_protospacer_region,218,37,18M1I6M,*,0,0,GCATCCATATCCATCCACAAACGAC,Null,Null
2,A00351:76:HCLKMDSXX:3:1104:19280:10520_4:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,Null,Null
3,A00351:76:HCLKMDSXX:3:1105:22752:32409_1:N:0:0...,16,Chr3_GTL1_protospacer_region,223,37,13M1I11M,*,0,0,CATATCCATCCACAAACGACGACCT,Null,Null
4,A00351:76:HCLKMDSXX:3:1105:22752:32409_4:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,Null,Null
...,...,...,...,...,...,...,...,...,...,...,...,...
2127,A00351:76:HCLKMDSXX:3:2673:13747:30373_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1486,37,25M,*,0,0,TGCATCCACATGTTTTTGATTCCTG,Null,Null
2128,A00351:76:HCLKMDSXX:3:2674:4906:12336_1:N:0:0A...,16,Chr3_GTL1_protospacer_region,1486,37,25M,*,0,0,TGCATCCACATGTTTTTGATTCCTG,Null,Null
2129,A00351:76:HCLKMDSXX:3:2674:23818:16720_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1260,37,25M,*,0,0,AACAGCCTGTGTGGTGGGCCCGGCA,Null,Null
2130,A00351:76:HCLKMDSXX:3:2675:21730:15640_1:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,Null,Null


In [32]:
for i in range(possible_indel_rows.shape[0]):
    finder_result = protospacer_finder(possible_indel_rows.iloc[i]['POS'],acceptable_index_range)
    assert_if_wt = unwanted_labels(possible_indel_rows.iloc[i]['CIGAR'])
    possible_indel_rows.at[i,'protospacer'] = finder_result
    possible_indel_rows.at[i,'CIGAR'] = assert_if_wt
    #taken the seq from the segments
    starting_PAM_index = specific_index_range[finder_result] - possible_indel_rows.iloc[i]['POS']
    if starting_PAM_index >= 0 and possible_indel_rows.iloc[i]['POS']+starting_PAM_index+10 < possible_indel_rows.iloc[i]['POS']+25:
        possible_indel_rows.at[i,'real_pam_seq'] = possible_indel_rows.iloc[i]['SEQ'][starting_PAM_index:starting_PAM_index+10]
        if assert_if_wt == 'wt':
            if possible_indel_rows.loc[i,'real_pam_seq'] != wt_bank[possible_indel_rows.loc[i,'protospacer']]:
                possible_indel_rows.at[i,'real_pam_seq'] = 'Null'
            else:
                possible_indel_rows.at[i,'real_pam_seq'] = 'wt'
possible_indel_rows

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq
0,A00351:76:HCLKMDSXX:3:1101:20338:25254_1:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,p1,CACAAACGAC
1,A00351:76:HCLKMDSXX:3:1104:19280:10520_1:N:0:0...,16,Chr3_GTL1_protospacer_region,218,37,18M1I6M,*,0,0,GCATCCATATCCATCCACAAACGAC,p1,Null
2,A00351:76:HCLKMDSXX:3:1104:19280:10520_4:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,p1,CACAAACGAC
3,A00351:76:HCLKMDSXX:3:1105:22752:32409_1:N:0:0...,16,Chr3_GTL1_protospacer_region,223,37,13M1I11M,*,0,0,CATATCCATCCACAAACGACGACCT,p1,CACAAACGAC
4,A00351:76:HCLKMDSXX:3:1105:22752:32409_4:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,p1,CACAAACGAC
...,...,...,...,...,...,...,...,...,...,...,...,...
2127,A00351:76:HCLKMDSXX:3:2673:13747:30373_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1486,37,wt,*,0,0,TGCATCCACATGTTTTTGATTCCTG,p7,wt
2128,A00351:76:HCLKMDSXX:3:2674:4906:12336_1:N:0:0A...,16,Chr3_GTL1_protospacer_region,1486,37,wt,*,0,0,TGCATCCACATGTTTTTGATTCCTG,p7,wt
2129,A00351:76:HCLKMDSXX:3:2674:23818:16720_1:N:0:0...,16,Chr3_GTL1_protospacer_region,1260,37,wt,*,0,0,AACAGCCTGTGTGGTGGGCCCGGCA,p4,wt
2130,A00351:76:HCLKMDSXX:3:2675:21730:15640_1:N:0:0...,0,Chr3_GTL1_protospacer_region,225,37,11M1I13M,*,0,0,TATCCATCCACAAACGACGACCTAA,p1,CACAAACGAC


In [17]:
a = possible_indel_rows.loc[possible_indel_rows['protospacer'] == 'p5']
a.loc[a['real_pam_seq'] == 'wt']

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,protospacer,real_pam_seq


counts_for_types = possible_indel_rows['real_pam_seq'].value_counts()
print(counts_for_types, type(counts_for_types))

In [33]:
#An example of a part of indel_count_from_sam_files_ordered_by_plants type of files
leaf_name = '1A'
indel_count_series = possible_indel_rows.groupby(['protospacer', 'real_pam_seq']).size()
new_df = indel_count_series.to_frame(name = leaf_name).reset_index()
new_df = new_df[new_df.real_pam_seq != 'Null']
new_df

,protospacer,real_pam_seq,1A
0,p1,AACAAACGAC,1
1,p1,CACAAACGAA,2
2,p1,CACAAACGAC,934
3,p1,CACAAATCGA,2
4,p1,CACAATCGAC,1
5,p1,CACAGACGAC,1
7,p1,TACAAACGAC,2
8,p1,wt,140
9,p2,GCTACCGTAC,2
11,p2,wt,147


In [34]:
possible_indel_rows.protospacer.unique()

array(['p1', 'p7', 'p4', 'p5', 'p2', 'p6', 'p3'], dtype=object)

In [35]:
#new_df.to_csv('1A_indel_count.csv', index = False)